In [1]:
%%capture
%run "EDA.ipynb"
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
# --- 1. PRÉPARATION (Identique au Random Forest) ---

# Standardisation de la colonne système
df_clean['system_simplified'] = 'other'
sys_lower = df_clean['crop_production_system'].str.lower()
df_clean.loc[sys_lower.str.contains('irrigated|water|dam|riverine', na=False), 'system_simplified'] = 'irrigated'
df_clean.loc[sys_lower.str.contains('rainfed|dieri|recessional', na=False), 'system_simplified'] = 'rainfed'
df_clean.loc[sys_lower.str.contains('commercial|mechanized|large_scale|lscf', na=False), 'system_simplified'] = 'commercial_mechanized'
df_clean.loc[sys_lower.str.contains('traditional|communal|small|pastoral|sscf|a1|a2', na=False), 'system_simplified'] = 'traditional_small_scale'
df_clean.loc[sys_lower.str.contains('all|none|or \(ps\)', na=False), 'system_simplified'] = 'general_unknown'


In [3]:
# Features et Target
features_cols = ['country_code', 'season_name', 'planting_month', 'harvest_month', 'area', 'system_simplified']
X = df_clean[features_cols]
y = df_clean['yield']

# Encodage One-Hot
X = pd.get_dummies(X, columns=['country_code', 'season_name', 'system_simplified'], drop_first=True)

# Division Train/Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:


# --- SPÉCIFIQUE RÉGRESSION LINÉAIRE : MISE À L'ÉCHELLE ---
# Les modèles linéaires "paniquent" si une variable est beaucoup plus grande 
# que les autres (Mois = 12). On utilise StandardScaler.
scaler = StandardScaler()
# On fit uniquement sur le train pour éviter la fuite de données
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- 2. OPTIMISATION (GRID SEARCH SUR RIDGE) ---
# Alpha = force de la régularisation (0 = régression linéaire simple, 100 = forte contrainte)
param_grid = {'alpha': [0.01, 0.1, 1, 10, 100]}

ridge = Ridge()
grid_search_lr = GridSearchCV(estimator=ridge, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1)

print("Optimisation de la Régression Linéaire (Ridge)...")
grid_search_lr.fit(X_train_scaled, y_train)

# --- 3. RÉSULTATS ---
best_lr = grid_search_lr.best_estimator_
y_pred_lr = best_lr.predict(X_test_scaled)

print(f"\nMeilleur Alpha : {grid_search_lr.best_params_}")
print(f"R² (Test)      : {r2_score(y_test, y_pred_lr):.4f}")
print(f"MAE (Test)     : {mean_absolute_error(y_test, y_pred_lr):.4f}")

Optimisation de la Régression Linéaire (Ridge)...

Meilleur Alpha : {'alpha': 10}
R² (Test)      : 0.3440
MAE (Test)     : 0.4737


In [5]:
# Sauvegarde du modèle
model_path = '../ml_models_pkg/ridge_regression_model.pkl'
joblib.dump(best_lr, model_path)
print(f"Modèle sauvegardé à : {model_path}")

# Sauvegarde du scaler également (nécessaire pour les prédictions futures)
scaler_path = '../ml_models_pkg/scaler.pkl'
joblib.dump(scaler, scaler_path)
print(f"Scaler sauvegardé à : {scaler_path}")

Modèle sauvegardé à : ../ml_models_pkg/ridge_regression_model.pkl
Scaler sauvegardé à : ../ml_models_pkg/scaler.pkl
